# Тестовое задание УЖЦК

В данном тестовом задании предлагается выполнить несколько прикладных задач, связанных с псевдоданными УЖЦК.

### Описание данных

В приложении к заданию есть файл database.db. Он представляет собой файл БД на фреймворке sqlite3. Синтаксис SQL, используемый в данной БД почти не отличается от ORACLE DB (более подробно можно почитать тут: https://unetway.com/tutorial/sqlite-syntax).
Запросы к базе можно сохранять сразу в pandas.Dataframe, используя следующий код:

In [1]:
import pandas as pd
import sqlite3 as sql

query = 'SELECT * FROM pl LIMIT 5'
con = sql.connect('database.db')
df = pd.read_sql_query(sql = query, con=con)
con.close()

df

,client_id,segment,period,product,pl
0,ZDosW1zUAi,Массовый,p_1,Кредит,9.018067
1,wVRHQ9hzrw,Массовый,p_1,Кредит,18.889585
2,6DZQIaOyL3,Массовый,p_1,Кредит,0.369257
3,jDwO0sMhL2,Массовый,p_1,Кредит,124.315319
4,FXIJUkgjLy,Массовый,p_1,Кредит,41.287831


   В базе данных хранится 2 таблицы: pl и campaign. Ключами для связи этих таблиц является поле client_id. В таблице pl находится информация о ЧОД (чистый операционный доход) от клиента с продуктом в разрезе периодов. В таблице campaign хранятся данные о прошедшей  между отчетными периодами "p_1" и "p_2" маркетинговой кампании.<br>
Атрибутный состав таблицы pl:
   - client_id - id клиента
   - segment - сегмент клиента внутри СМБ(Средний и малый бизнес)
   - period - период расчета ЧОД банка от продукта, используемого клиентом
   - product - продукт 
   - pl - ЧОД в тысячах рублей
   
Пример: строка 0 на срезе выше - в периоде "p_1" за продукт "Кредит" с клиента "yxwtZv530y" массового сегмента банк заработал 0.374924 тыс. рублей.
   
Если в каком-либо периоде информация о клиенте/продукте у клиента отсутствует, значит в этот период клиента в банке не было/клиент продуктом не пользовался.

Атрибутный состав таблицы campaign:
- client_id - id клиента
- product - продукт кампании
- response - статус отклика на предложение
- manager_id - id менеджера, на которого назначен клиент 

### Задание 

1) Посчитать средний ЧОД по продуктам и количество клиентов с продуктом для каждого сегмента в разрезе периодов.<br>
2) Посчитать средний ЧОД и количество клиентов в каждом сегменте в разрезе периодов.<br>
3) Оценить отток клиентов с продуктов в периоде p_2.<br>
4) Посчитать сумму ЧОД, привлеченную каждым менеджером в ходе маркетинговой кампании<br>
5) Оценить эффективность маркетинговой кампании.

Необходимо ознакомиться с данными.

In [2]:
query_pl = 'SELECT * FROM pl'
con = sql.connect('database.db')
pl_table = pd.read_sql_query(sql = query_pl, con=con)
con.close()

pl_table

,client_id,segment,period,product,pl
0,ZDosW1zUAi,Массовый,p_1,Кредит,9.018067
1,wVRHQ9hzrw,Массовый,p_1,Кредит,18.889585
2,6DZQIaOyL3,Массовый,p_1,Кредит,0.369257
3,jDwO0sMhL2,Массовый,p_1,Кредит,124.315319
4,FXIJUkgjLy,Массовый,p_1,Кредит,41.287831
...,...,...,...,...,...
92604,G8E6dyKT3k,Массовый,p_2,Банковская карта,17.362040
92605,10OPeJCF1l,Массовый,p_2,Банковская карта,1.861434
92606,BMN762bZh5,Верхний,p_2,Банковская карта,78.430698
92607,FcB9pJHGVX,Стандартный,p_2,Банковская карта,16.107313


Проверка на пропуски в данных.

In [3]:
pl_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92609 entries, 0 to 92608
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   client_id  92609 non-null  object 
 1   segment    92609 non-null  object 
 2   period     92609 non-null  object 
 3   product    92609 non-null  object 
 4   pl         92609 non-null  float64
dtypes: float64(1), object(4)
memory usage: 3.5+ MB


In [4]:
pl_table.isnull().sum()

client_id    0
segment      0
period       0
product      0
pl           0
dtype: int64

In [5]:
pd.options.display.float_format ='{:,.6f}'.format

In [6]:
pl_table.describe()

,pl
count,"92,609.000000"
mean,30.236802
std,58.209884
min,0.000000
25%,2.431644
50%,9.145721
75%,28.123301
max,527.538254


Посмотрим какие есть категориальные переменные.

In [7]:
pl_table.segment.unique()

array(['Массовый', 'Стандартный', 'Верхний'], dtype=object)

In [8]:
pl_table.period.unique()

array(['p_1', 'p_2'], dtype=object)

In [9]:
pl_table.groupby('product').product.count()


product
Банковская карта     5002
Депозит             11281
Кредит              18742
РКО                 47516
Эквайринг           10068
Name: product, dtype: int64

Аналогично со второй таблицей.

In [10]:
query_campaign = 'SELECT * FROM campaign'
con = sql.connect('database.db')
campaign_table = pd.read_sql_query(sql = query_campaign, con=con)
con.close()

campaign_table

,client_id,product,response,manager_id
0,owOyJEIst9,Кредит,Согласие,CM_EiBeeRMhdA
1,HkhuPN1KG7,Кредит,Не обработано менеджером,CM_PzEEav4ic0
2,x1rEdC5fys,Кредит,Согласие,CM_RCwv1r0OHO
3,fx0acVADvj,Кредит,Согласие,CM_nJ7j5pAcMB
4,nDNNydkKQR,Кредит,Согласие,CM_EiBeeRMhdA
...,...,...,...,...
4995,pbL8qc9JQJ,Кредит,Отказ,CM_doxY9cvELI
4996,0u4KYTNs7g,Кредит,Не обработано менеджером,CM_9vMKq9aqw9
4997,forPIbFbFc,Кредит,Отказ,CM_eE0WMK1vVW
4998,TEHS2F9oXq,Кредит,Отказ,CM_nJ7j5pAcMB


In [11]:
campaign_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   client_id   5000 non-null   object
 1   product     5000 non-null   object
 2   response    5000 non-null   object
 3   manager_id  5000 non-null   object
dtypes: object(4)
memory usage: 156.4+ KB


In [12]:
campaign_table.isnull().sum()

client_id     0
product       0
response      0
manager_id    0
dtype: int64

In [13]:
campaign_table.groupby('product').product.count()

product
Кредит    5000
Name: product, dtype: int64

In [14]:
campaign_table.response.unique()

array(['Согласие', 'Не обработано менеджером', 'Недозвон', 'Перезвонить',
       'Отказ'], dtype=object)

### 1) Посчитать средний ЧОД по продуктам и количество клиентов с продуктом для каждого сегмента в разрезе периодов.

In [15]:
pl_table

,client_id,segment,period,product,pl
0,ZDosW1zUAi,Массовый,p_1,Кредит,9.018067
1,wVRHQ9hzrw,Массовый,p_1,Кредит,18.889585
2,6DZQIaOyL3,Массовый,p_1,Кредит,0.369257
3,jDwO0sMhL2,Массовый,p_1,Кредит,124.315319
4,FXIJUkgjLy,Массовый,p_1,Кредит,41.287831
...,...,...,...,...,...
92604,G8E6dyKT3k,Массовый,p_2,Банковская карта,17.362040
92605,10OPeJCF1l,Массовый,p_2,Банковская карта,1.861434
92606,BMN762bZh5,Верхний,p_2,Банковская карта,78.430698
92607,FcB9pJHGVX,Стандартный,p_2,Банковская карта,16.107313


In [16]:
pl_table.segment.unique()

array(['Массовый', 'Стандартный', 'Верхний'], dtype=object)

In [17]:
# Группируем данные по продукту и подсчитываем среднее значение ЧОД
mean_all_pl = pl_table.groupby('product').mean('pl')
mean_all_pl.columns = ['mean_all_pl_agg']

# Группируем данные по продукту и подсчитываем количество клиентов с продуктом
count_all_clients = pl_table.groupby('product').count()[['client_id']]
count_all_clients.columns = ['count_all_clients_agg']

# Фильтруем данные по периоду p_1 и массовому сегменту, группируем по продукту и высчитываем среднее значение ЧОД
p1_mass_pl = pl_table[(pl_table['period'] == 'p_1') & (
    pl_table['segment'] == 'Массовый')].groupby('product').mean('pl')
p1_mass_pl.columns = ['p1_mass_mean_pl']

# Фильтруем данные по периоду p_1 и сегменту 'Стандартный', группируем по продукту и высчитываем среднее значение ЧОД
p1_st_pl = pl_table[(pl_table['period'] == 'p_1') & (
    pl_table['segment'] == 'Стандартный')].groupby('product').mean('pl')
p1_st_pl.columns = ['p1_st_mean_pl']

#  Фильтруем данные по периоду p_1 и сегменту 'Верхний', группируем по продукту и высчитываем среднее значение ЧОД
p1_top_pl = pl_table[(pl_table['period'] == 'p_1') & (
    pl_table['segment'] == 'Верхний')].groupby('product').mean('pl')
p1_top_pl.columns = ['p1_top_mean_pl']

# Фильтруем данные по периоду p_2 и сегменту 'Массовый', группируем по продукту и высчитываем среднее значение ЧОД
p2_mass_pl = pl_table[(pl_table['period'] == 'p_2') & (
    pl_table['segment'] == 'Массовый')].groupby('product').mean('pl')
p2_mass_pl.columns = ['p2_mass_mean_pl']

# Фильтруем данные по периоду p_2 и сегменту 'Стандартный', группируем по продукту и высчитываем среднее значение ЧОД
p2_st_pl = pl_table[(pl_table['period'] == 'p_2') & (
    pl_table['segment'] == 'Стандартный')].groupby('product').mean('pl')
p2_st_pl.columns = ['p2_st_mean_pl']

# Фильтруем данные по периоду p_2 и сегменту 'Верхний', группируем по продукту и высчитываем среднее значение ЧОД
p2_top_pl = pl_table[(pl_table['period'] == 'p_2') & (
    pl_table['segment'] == 'Верхний')].groupby('product').mean('pl')
p2_top_pl.columns = ['p2_top_mean_pl']

# Фильтруем данные по периоду p_1 и сегменту 'Массовый', группируем по продукту и подсчитываем количество клиентов
p1_mass_clients = pl_table[(pl_table['period'] == 'p_1') & (
    pl_table['segment'] == 'Массовый')].groupby('product').count()[['client_id']]
p1_mass_clients.columns = ['p1_mass_counted_clients']

# Фильтруем данные по периоду p_1 и сегменту 'Стандартный', группируем по продукту и подсчитываем количество клиентов
p1_st_clients = pl_table[(pl_table['period'] == 'p_1') & (
    pl_table['segment'] == 'Стандартный')].groupby('product').count()[['client_id']]
p1_st_clients.columns = ['p1_st_counted_clients']

# Фильтруем данные по периоду p_1 и сегменту 'Верхний', группируем по продукту и подсчитываем количество клиентов
p1_top_clients = pl_table[(pl_table['period'] == 'p_1') & (
    pl_table['segment'] == 'Верхний')].groupby('product').count()[['client_id']]
p1_top_clients.columns = ['p1_top_counted_clients']

# Фильтруем данные по периоду p_2 и сегменту 'Массовый', группируем по продукту и подсчитываем количество клиентов
p2_mass_clients = pl_table[(pl_table['period'] == 'p_2') & (
    pl_table['segment'] == 'Массовый')].groupby('product').count()[['client_id']]
p2_mass_clients.columns = ['p2_mass_counted_clients']

# Фильтруем данные по периоду p_2 и сегменту 'Стандартный', группируем по продукту и подсчитываем количество клиентов
p2_st_clients = pl_table[(pl_table['period'] == 'p_2') & (
    pl_table['segment'] == 'Стандартный')].groupby('product').count()[['client_id']]
p2_st_clients.columns = ['p2_st_counted_clients']

# Фильтруем данные по периоду p_2 и сегменту 'Верхний', группируем по продукту и подсчитываем количество клиентов
p2_top_clients = pl_table[(pl_table['period'] == 'p_2') & (
    pl_table['segment'] == 'Верхний')].groupby('product').count()[['client_id']]
p2_top_clients.columns = ['p2_top_counted_clients']

# Соединяем все данные в одну таблицу
task_1 = pd.concat([mean_all_pl, count_all_clients, p1_mass_pl, p1_st_pl, p1_top_pl, p2_mass_pl, p2_st_pl, p2_top_pl,
                    p1_mass_clients, p1_st_clients, p1_top_clients, p2_mass_clients, p2_st_clients, p2_top_clients], 
                   axis=1)

'''

Колонки "mean_all_pl_agg" и "count_all_clients_agg" обобщают все данные без среза по периодам и сегментам

Колонки закодированы следующим образом: 

p1,p2 - отчетные периоды "p_1" и "p_2"
mass, st, top - соответственно массовый, стандартный и верхний сегмент
mean_pl - среднее значение ЧОД
counted_clients - количество клиентов

Например: p1_st_mean_pl

'''

task_1

,mean_all_pl_agg,count_all_clients_agg,p1_mass_mean_pl,p1_st_mean_pl,p1_top_mean_pl,p2_mass_mean_pl,p2_st_mean_pl,p2_top_mean_pl,p1_mass_counted_clients,p1_st_counted_clients,p1_top_counted_clients,p2_mass_counted_clients,p2_st_counted_clients,p2_top_counted_clients
product,,,,,,,,,,,,,,
Банковская карта,16.781657,5002,13.310203,26.421616,60.571545,11.726557,22.436106,51.770009,895,338,49,2636,917,167
Депозит,37.600983,11281,28.747545,60.994262,112.474520,25.219855,49.332790,99.260623,3537,1292,254,4345,1570,283
Кредит,88.628434,18742,59.219407,120.101219,236.938710,67.109774,131.611788,248.191513,5247,1871,360,7845,2871,548
РКО,7.865137,47516,5.795114,11.418812,22.847632,5.566726,10.983568,22.064079,16728,5907,1149,16700,5870,1162
Эквайринг,25.554980,10068,19.627608,39.329551,80.517184,17.348344,35.046014,72.968866,2625,962,183,4424,1590,284


### 2) Посчитать средний ЧОД и количество клиентов в каждом сегменте в разрезе периодов.

In [18]:

# Группируем данные по 'segment' и подсчитываем среднее значение ЧОД
mean_segments = pl_table.groupby('segment').mean('pl')
mean_segments.columns = ['mean_pl_segm']

# Группируем данные по 'segment' и подсчитываем количество клиентов
counted_clients = pl_table.groupby('segment').count()[['client_id']]
counted_clients.columns = ['counted_all_clients_agg']

#  Фильтруем данные по периоду p_1, группируем по 'segment' и высчитываем среднее значение ЧОД
p1_pl = pl_table[(pl_table['period'] == 'p_1')].groupby('segment').mean('pl')
p1_pl.columns = ['p1_mean_pl']

#  Фильтруем данные по периоду p_2, группируем по 'segment' и высчитываем среднее значение ЧОД
p2_pl = pl_table[(pl_table['period'] == 'p_2')].groupby('segment').mean('pl')
p2_pl.columns = ['p2_mean_pl']

# Фильтруем данные по периоду p_1, группируем по 'segment' и подсчитываем количество клиентов
p1_clients = pl_table[(pl_table['period'] == 'p_1')].groupby('segment').count()[['client_id']]
p1_clients.columns = ['p1_counted_clients']

# Фильтруем данные по периоду p_2, группируем по 'segment' и подсчитываем количество клиентов
p2_clients = pl_table[(pl_table['period'] == 'p_2')].groupby('segment').count()[['client_id']]
p2_clients.columns = ['p2_counted_clients']

# Соединяем все данные в одну таблицу
task_2 = pd.concat([mean_segments, counted_clients, p1_pl, p2_pl, p1_clients, p2_clients], 
                   axis=1)

'''

Колонки "mean_pl_segm" и "counted_all_clients_agg" обобщают все данные без среза по периодам

Колонки закодированы следующим образом: 

p1,p2 - отчетные периоды "p_1" и "p_2"
mean_pl - среднее значение ЧОД
counted_clients - количество клиентов

Например: p1_mean_pl

'''


task_2

,mean_pl_segm,counted_all_clients_agg,p1_mean_pl,p2_mean_pl,p1_counted_clients,p2_counted_clients
segment,,,,,,
Верхний,84.912809,4439,79.108293,89.650947,1995,2444
Массовый,21.690023,64982,19.729266,23.273464,29032,35950
Стандартный,43.721363,23188,40.282587,46.503397,10370,12818


### 3) Оценить отток клиентов с продуктов в периоде p_2.

In [19]:
# Группируем по 'product' и подсчитываем количество клиентов без учета периодов
count_all_clients = pl_table.groupby('product').count()[['client_id']]
count_all_clients.columns = ['counted_all_clients']

# Группируем по 'product' и подсчитываем количество клиентов за период p_1
clients_P1 = pl_table[(pl_table['period'] == 'p_1')].groupby('product').count()[['client_id']]
clients_P1.columns = ['counted_clients_P1']

# Группируем по 'product' и подсчитываем количество клиентов за период p_2
clients_P2 = pl_table[(pl_table['period'] == 'p_2')].groupby('product').count()[['client_id']]
clients_P2.columns = ['counted_clients_P2']

# Соединяем таблицы в один датафрейм
task_3 = pd.concat([count_all_clients, clients_P1, clients_P2], 
                   axis=1)
task_3

,counted_all_clients,counted_clients_P1,counted_clients_P2
product,,,
Банковская карта,5002,1282,3720
Депозит,11281,5083,6198
Кредит,18742,7478,11264
РКО,47516,23784,23732
Эквайринг,10068,3770,6298


Считаем приток или отток клиентов в периоде P2 в сравнении с периодом P1. 

In [20]:
task_3['p2_minus_p1'] = task_3['counted_clients_P2'] - task_3['counted_clients_P1']
task_3['p2_minus_p1_%'] = ((task_3['counted_clients_P2'] - task_3['counted_clients_P1'])/task_3['counted_clients_P2']) * 100
task_3

,counted_all_clients,counted_clients_P1,counted_clients_P2,p2_minus_p1,p2_minus_p1_%
product,,,,,
Банковская карта,5002,1282,3720,2438,65.537634
Депозит,11281,5083,6198,1115,17.989674
Кредит,18742,7478,11264,3786,33.611506
РКО,47516,23784,23732,-52,-0.219113
Эквайринг,10068,3770,6298,2528,40.139727


Есть небольшой отток клиентов в продукте "РКО" - 52 клиента или 0,2 %. По остальным продуктам наблюдается прирост.

Попробуем аналогично посчитать как изменился ЧОД во втором отчетном периоде.

In [21]:
sum_all_pl = pl_table.groupby('product').sum()[['pl']]
sum_all_pl.columns = ['sum_pl']

sum_P1 = pl_table[(pl_table['period'] == 'p_1')].groupby('product').sum()[['pl']]
sum_P1.columns = ['sum_P1']

sum_P2 = pl_table[(pl_table['period'] == 'p_2')].groupby('product').sum()[['pl']]
sum_P2.columns = ['sum_P2']

task_3_2 = pd.concat([sum_all_pl, sum_P1, sum_P2], 
                   axis=1)
task_3_2


,sum_pl,sum_P1,sum_P2
product,,,
Банковская карта,"83,941.847177","23,811.143590","60,130.703587"
Депозит,"424,176.687118","209,053.182401","215,123.504717"
Кредит,"1,661,074.117436","620,731.548140","1,040,342.569297"
РКО,"373,719.833676","190,643.509100","183,076.324576"
Эквайринг,"257,287.536140","104,092.143415","153,195.392725"


Проверим были ли потери в ЧОДе.

In [22]:
task_3_2['p2_minus_p1'] = task_3_2['sum_P2'] - task_3_2['sum_P1']
task_3_2['p2_minus_p1_%'] = ((task_3_2['sum_P2'] - task_3_2['sum_P1'])/task_3_2['sum_P2']) * 100
task_3_2

,sum_pl,sum_P1,sum_P2,p2_minus_p1,p2_minus_p1_%
product,,,,,
Банковская карта,"83,941.847177","23,811.143590","60,130.703587","36,319.559996",60.401023
Депозит,"424,176.687118","209,053.182401","215,123.504717","6,070.322317",2.821785
Кредит,"1,661,074.117436","620,731.548140","1,040,342.569297","419,611.021157",40.333928
РКО,"373,719.833676","190,643.509100","183,076.324576","-7,567.184524",-4.133350
Эквайринг,"257,287.536140","104,092.143415","153,195.392725","49,103.249310",32.052693


В том же продукте "РКО" можно наблюдать уменьшение дохода в P2 на 4.1% .

### 4) Посчитать сумму ЧОД, привлеченную каждым менеджером в ходе маркетинговой кампании

In [23]:
campaign_table.response.unique()

array(['Согласие', 'Не обработано менеджером', 'Недозвон', 'Перезвонить',
       'Отказ'], dtype=object)

In [24]:
campaign_table.groupby('product').product.count()

product
Кредит    5000
Name: product, dtype: int64

Будем производить рассчет по данным, где от клиента получено согласие и продуктом является кредит.

In [25]:
# Отфильтруем данные по продукту "Кредит", сгруппируем по клиентам и суммируем ЧОД
sum_on_id = pl_table[(pl_table['product'] == 'Кредит')].groupby('client_id').sum()[['pl']]
sum_on_id

,pl
client_id,
00JURPZFUt,155.970455
00USCNy9Xl,53.416680
01EGcbCHCY,56.850698
01hLVZzHfy,79.819914
01v8qZNp3S,50.464543
...,...
zzO1OyFv82,20.948042
zziCb9JaPT,191.443573
zzk7UXy7zD,134.761386


Если клиент обслуживается только одним менеджером и попадает в таблицу 'pl' только после привлечения маркетинговой кампании, то можно сделать следующее объединение данных.

In [26]:
task_4 = campaign_table[(campaign_table['response'] == 'Согласие')].merge(sum_on_id, on='client_id', how='inner')
task_4

,client_id,product,response,manager_id,pl
0,owOyJEIst9,Кредит,Согласие,CM_EiBeeRMhdA,27.660845
1,x1rEdC5fys,Кредит,Согласие,CM_RCwv1r0OHO,80.326489
2,fx0acVADvj,Кредит,Согласие,CM_nJ7j5pAcMB,39.234655
3,nDNNydkKQR,Кредит,Согласие,CM_EiBeeRMhdA,6.988138
4,6q0ZBvQX5z,Кредит,Согласие,CM_RCwv1r0OHO,2.227366
...,...,...,...,...,...
2811,qdtEeaguxa,Кредит,Согласие,CM_wjb2ArrpAw,12.091074
2812,OqRATwr16M,Кредит,Согласие,CM_gHP4ykbzTj,350.230937
2813,5bVDUHXKT3,Кредит,Согласие,CM_doxY9cvELI,125.817001
2814,4701Nvidi2,Кредит,Согласие,CM_c28lGRRaEI,146.875239


Проверим сколько менеджеров в выборке.

In [27]:
campaign_table.manager_id.unique()

array(['CM_EiBeeRMhdA', 'CM_PzEEav4ic0', 'CM_RCwv1r0OHO', 'CM_nJ7j5pAcMB',
       'CM_2755B4KTu4', 'CM_zShYeVNFi0', 'CM_fQHTUUUkXc', 'CM_gHP4ykbzTj',
       'CM_onY6ha8qHv', 'CM_R7BdTh0Svt', 'CM_c28lGRRaEI', 'CM_eE0WMK1vVW',
       'CM_70HjN0EVqf', 'CM_6ofody41cY', 'CM_wjb2ArrpAw', 'CM_pMFgOhtUKD',
       'CM_lKxhD73Z6d', 'CM_doxY9cvELI', 'CM_dcTdLjqH2x', 'CM_9vMKq9aqw9'],
      dtype=object)

Сгруппируем данные по менеджерам, суммируем ЧОД и сортируем значения.

In [28]:
task_4 = task_4.groupby('manager_id').sum()[['pl']].sort_values(by='pl', ascending=False)
task_4

,pl
manager_id,
CM_2755B4KTu4,"16,600.098122"
CM_wjb2ArrpAw,"14,136.285720"
CM_R7BdTh0Svt,"14,119.009958"
CM_gHP4ykbzTj,"14,106.604720"
CM_pMFgOhtUKD,"13,543.156752"
CM_6ofody41cY,"13,407.509851"
CM_c28lGRRaEI,"13,358.891157"
CM_eE0WMK1vVW,"13,298.954882"
CM_PzEEav4ic0,"12,933.254512"


In [29]:
task_4.describe()

,pl
count,20.000000
mean,"12,802.205898"
std,"1,344.698581"
min,"11,080.970431"
25%,"11,823.049894"
50%,"12,812.184737"
75%,"13,441.421576"
max,"16,600.098122"


### 5) Оценить эффективность маркетинговой кампании.

Посчитаем общий ЧОД маркетинговой кампании.

In [30]:
task_4['pl'].sum()

256044.11795058392

In [31]:
# Отфильтруем данные по продукту "Кредит", сгруппируем по клиентам и суммируем ЧОД
sum_on_id_ = pl_table[(pl_table['product'] == 'Кредит')].groupby('client_id').sum()[['pl']]

# Соединим с данными из маркетинговой кампании, где от клиентов получено согласие
task_5 = campaign_table[(campaign_table['response'] == 'Согласие')].merge(sum_on_id_, on='client_id', how='inner')
task_5

,client_id,product,response,manager_id,pl
0,owOyJEIst9,Кредит,Согласие,CM_EiBeeRMhdA,27.660845
1,x1rEdC5fys,Кредит,Согласие,CM_RCwv1r0OHO,80.326489
2,fx0acVADvj,Кредит,Согласие,CM_nJ7j5pAcMB,39.234655
3,nDNNydkKQR,Кредит,Согласие,CM_EiBeeRMhdA,6.988138
4,6q0ZBvQX5z,Кредит,Согласие,CM_RCwv1r0OHO,2.227366
...,...,...,...,...,...
2811,qdtEeaguxa,Кредит,Согласие,CM_wjb2ArrpAw,12.091074
2812,OqRATwr16M,Кредит,Согласие,CM_gHP4ykbzTj,350.230937
2813,5bVDUHXKT3,Кредит,Согласие,CM_doxY9cvELI,125.817001
2814,4701Nvidi2,Кредит,Согласие,CM_c28lGRRaEI,146.875239


Перепроверяем сумму ЧОД.

In [32]:
task_5.pl.sum()

256044.11795058392

Считаем количество привлеченных клиентов.

In [33]:
task_5.client_id.count()

2816

Смотрим какое количество клиентов и какой ЧОД были привлечены в периодах P1 и P2.

In [34]:
sum_on_id_p1 = pl_table[(pl_table['period'] == 'p_1') & 
                        (pl_table['product'] == 'Кредит')].groupby('client_id').sum()[['pl']]

task_5_p1 = campaign_table[(campaign_table['response'] == 'Согласие')].merge(sum_on_id_p1, on='client_id', how='inner')


task_5_p1

,client_id,product,response,manager_id,pl


In [35]:
sum_on_id_p2 = pl_table[(pl_table['period'] == 'p_2') & 
                        (pl_table['product'] == 'Кредит')].groupby('client_id').sum()[['pl']]

task_5_p2 = campaign_table[(campaign_table['response'] == 'Согласие')].merge(sum_on_id_p2, on='client_id', how='inner')

task_5_p2

,client_id,product,response,manager_id,pl
0,owOyJEIst9,Кредит,Согласие,CM_EiBeeRMhdA,27.660845
1,x1rEdC5fys,Кредит,Согласие,CM_RCwv1r0OHO,80.326489
2,fx0acVADvj,Кредит,Согласие,CM_nJ7j5pAcMB,39.234655
3,nDNNydkKQR,Кредит,Согласие,CM_EiBeeRMhdA,6.988138
4,6q0ZBvQX5z,Кредит,Согласие,CM_RCwv1r0OHO,2.227366
...,...,...,...,...,...
2811,qdtEeaguxa,Кредит,Согласие,CM_wjb2ArrpAw,12.091074
2812,OqRATwr16M,Кредит,Согласие,CM_gHP4ykbzTj,350.230937
2813,5bVDUHXKT3,Кредит,Согласие,CM_doxY9cvELI,125.817001
2814,4701Nvidi2,Кредит,Согласие,CM_c28lGRRaEI,146.875239


Эффект от маркетинговой кампании (МА) был только в периоде P2.

Перепроверяем итоговый ЧОД МА за период P2.

In [36]:
task_5_p2.pl.sum()

256044.11795058392

Рассчитаем отдельно по сегментам эффект от маркетинговой кампании.

In [37]:
task_5_segment = pl_table[(pl_table['product'] == 'Кредит')].merge(task_5_p2[['client_id']], on='client_id', how='inner')
task_5_segment

,client_id,segment,period,product,pl
0,XHgfHt3boG,Массовый,p_2,Кредит,35.244275
1,faWYjccoeJ,Массовый,p_2,Кредит,6.691619
2,9k7VeLIbKU,Массовый,p_2,Кредит,22.162824
3,GjqWSnMK1Y,Стандартный,p_2,Кредит,41.740930
4,o63q9tMgDW,Стандартный,p_2,Кредит,100.263930
...,...,...,...,...,...
2811,21sfo5BfbR,Стандартный,p_2,Кредит,163.617451
2812,lmilntvZFu,Массовый,p_2,Кредит,2.937910
2813,3zunWyDDE7,Массовый,p_2,Кредит,91.145467
2814,sEMCVeiw9F,Стандартный,p_2,Кредит,202.254485


In [38]:
task_5_segment_count = task_5_segment.groupby('segment').count()
task_5_segment_count = task_5_segment_count[['client_id']]
task_5_segment_count.columns = ['counted_clients']
task_5_segment_count

,counted_clients
segment,
Верхний,141
Массовый,1946
Стандартный,729


In [39]:
task_5_segment_count.sum()

counted_clients    2816
dtype: int64

In [40]:
task_5_segment_pl = task_5_segment.groupby('segment').sum()[['pl']]
task_5_segment_pl

,pl
segment,
Верхний,"37,378.877553"
Массовый,"125,371.315870"
Стандартный,"93,293.924527"


In [41]:
task_5_segment_pl.sum()

pl   256,044.117951
dtype: float64

#### Итого маркетинговая кампания привлекла

ЧОД в размере 256 044 117 руб, из которых по сегментам:
* Верхний - 37 378 877 руб
* Массовый - 125 371 315 руб
* Стандартный - 93 293 924 руб

2 816 клиентов, из которых по сегментам:
* Верхний - 141 клиент
* Массовый - 1946 клиентов
* Стандартный - 729 клиентов